## WGAN ##

In [1]:
import torch
import torchvision
from torchvision import utils
from torch.utils.data import DataLoader
from torch import nn
from torch.autograd import Variable
from pytorch_gan_metrics import get_inception_score
from tqdm import tqdm
import os
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [2]:
def create_CIFAR10_dataloaders(batch_size): 
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(), 
        torchvision.transforms.Resize(32), 
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    train_CIFAR10_set = torchvision.datasets.CIFAR10(root='./cifar10/', train=True, download=True, transform=transform)
    test_CIFAR10_set = torchvision.datasets.CIFAR10(root='./cifar10/', train=False, download=True, transform=transform)

    train_CIFAR10_dataloader = DataLoader(train_CIFAR10_set, batch_size=batch_size, shuffle=True, drop_last=True)
    test_CIFAR10_dataloader = DataLoader(test_CIFAR10_set, batch_size=batch_size, shuffle=True, drop_last=True)
    return train_CIFAR10_dataloader, test_CIFAR10_dataloader

print("Downloading CIFAR10 dataset...")
batch_size = 64
train_dataloader, test_dataloader = create_CIFAR10_dataloaders(batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Generator_WGAN(nn.Module):
    def __init__(self):
        super(Generator_WGAN, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.net = nn.Sequential(
            *block(100, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, 3*32*32),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.net(z)
        img = img.view(img.shape[0], *(3, 32, 32))
        return img

class Discriminator_WGAN(nn.Module):
    def __init__(self):
        super(Discriminator_WGAN, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(3*32*32, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

print("Instantiating DCGAN generator and discriminator...")
wgan_generator = Generator_WGAN()
wgan_discriminator = Discriminator_WGAN()
wgan_generator.to(device)
wgan_discriminator.to(device)
print()

Instantiating DCGAN generator and discriminator...



In [4]:
learning_rate=5e-4
epochs=50
batch_size=64
n_critic = 5
weight_cliping_limit=0.01

def train(generator, discriminator, train_dataloader):
    optim_generator = torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.9))
    optim_discriminator = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.9))

    if not os.path.exists('train_generated_images_wgan/'): 
        os.makedirs('train_generated_images_wgan')
        
    inception_score_file = open("inception_score_wgan.csv", "w")
    inception_score_file.write('epoch, inception_score \n')

    for epoch in tqdm(range(epochs)): 
        for i, (images, _) in enumerate(train_dataloader):

            real_images = Variable(images.type(torch.cuda.FloatTensor))

            ### train discriminator

            optim_discriminator.zero_grad()
            z = Variable(torch.Tensor(np.random.normal(0, 1, (images.shape[0], 100)))).to(device)
            fake_images = generator(z).detach()
            disc_loss = -torch.mean(discriminator(real_images)) + torch.mean(discriminator(fake_images))
            disc_loss.backward()
            optim_discriminator.step()

            # apply weight clipping
            for p in discriminator.parameters():
                p.data.clamp_(-weight_cliping_limit, weight_cliping_limit)

            # Train generator every n_critic batches
            if i % n_critic == 0:

                optim_generator.zero_grad()
                fake_images = generator(z)
                gen_loss = -torch.mean(discriminator(fake_images))
                gen_loss.backward()
                optim_generator.step()

        # compute inception score and samples every epoch
        z = Variable(torch.cuda.FloatTensor(np.random.normal(0, 1, (images.shape[0], 100)))).to(device)
        samples = generator(z)

        # normalize to [0, 1]
        samples = samples.mul(0.5).add(0.5)
        
        assert 0 <= samples.min() and samples.max() <= 1
        inception_score, inception_score_std = get_inception_score(samples)
        print("epoch: " + str(epoch) + ', inception score: ' + str(round(inception_score, 2)) + ' ± ' + str(round(inception_score_std, 2)))

        samples = samples[:64].data.cpu()
        grid = utils.make_grid(samples)
        utils.save_image(grid, 'train_generated_images_wgan/epoch_{}.png'.format(str(epoch)))
        
        inception_score_file.write(str(epoch) + ', ' + str(round(inception_score, 2)) + '\n')

    inception_score_file.close()

In [5]:
# train WGAN
print("training WGAN model...")
train(wgan_generator, wgan_discriminator, train_dataloader)

# save WGAN to file
#print("saving WGAN model to file...")
#torch.save(wgan_generator.state_dict(), 'wgan_generator.pkl')
#torch.save(wgan_discriminator.state_dict(), 'wgan_discriminator.pkl')

training WGAN model...


  2%|▏         | 1/50 [00:17<14:21, 17.58s/it]

epoch: 0, inception score: 1.63 ± 0.14


  4%|▍         | 2/50 [00:26<10:09, 12.69s/it]

epoch: 1, inception score: 1.74 ± 0.13


...

epoch: 47, inception score: 2.11 ± 0.46


 98%|█████████▊| 49/50 [08:33<00:09,  9.88s/it]

epoch: 48, inception score: 1.92 ± 0.3


100%|██████████| 50/50 [08:42<00:00, 10.45s/it]

epoch: 49, inception score: 1.84 ± 0.22
saving WGAN model to file...


In [6]:
def generate_images(generator):
    z = Variable(torch.cuda.FloatTensor(np.random.normal(0, 1, (batch_size, 100))))
    samples = generator(z)
    samples = samples.mul(0.5).add(0.5)
    samples = samples.data.cpu()
    grid = utils.make_grid(samples)
    print("Grid of 8x8 images saved to 'wgan_generated_images.png'.")
    utils.save_image(grid, 'wgan_generated_images.png')

def load_model(model, model_filename): 
    model.load_state_dict(torch.load(model_filename))

# load trained model and generate sample images
print("loading WGAN model...")
load_model(wgan_generator, 'wgan_generator.pkl')
load_model(wgan_discriminator, 'wgan_discriminator.pkl')

generate_images(wgan_generator)

loading WGAN model...
Grid of 8x8 images saved to 'wgan_generated_images.png'.
